In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("andrewmvd/medical-mnist")

print("Path to dataset files:", path)

/Users/I569354/Desktop/Praxisarbeiten/Studienarbeit_Sammlung/MedicaldeepLearning/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/I569354/Desktop/Praxisarbeiten/Studienarbeit_Sammlung/MedicaldeepLearning/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 84.8M/84.8M [00:07<00:00, 11.9MB/s]

Extracting files...


Path to dataset files: /Users/I569354/.cache/kagglehub/datasets/andrewmvd/medical-mnist/versions/1


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import medmnist
from medmnist import INFO, Evaluator

In [7]:
def download_dataset(data_flag):
  print(f"Dataset source information : MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

  download = True
  
  info = INFO[data_flag]

  DataClass = getattr(medmnist, info['python_class'])

  print('\nBreastMNIST')
  print(info['description'])

  # Downloading dataset
  train_dataset = DataClass(split='train', download=download)
  val_dataset = DataClass(split='val', download=download)
  test_dataset = DataClass(split='test', download=download)

  print('\nTraining Dataset:')
  print(train_dataset)
  # print(train_dataset.imgs.shape)
  if train_dataset.imgs.ndim == 3:
    print('Adding channel to images...')
    train_dataset.imgs = np.expand_dims(train_dataset.imgs, axis=-1)

  print('\nValidation Dataset:')
  print(val_dataset)
  # print(val_dataset.imgs.shape)
  if val_dataset.imgs.ndim == 3:
    print('Adding channel to images...')
    val_dataset.imgs = np.expand_dims(val_dataset.imgs, axis=-1)

  print('\nTesting Dataset:')
  print(test_dataset)
  # print(test_dataset.imgs.shape)
  if test_dataset.imgs.ndim == 3:
    print('Adding channel to images...')
    test_dataset.imgs = np.expand_dims(test_dataset.imgs, axis=-1)

  print('\nShapes of images:')
  print('Training: ', train_dataset.imgs.shape)
  print('Validation: ', val_dataset.imgs.shape)
  print('Testing: ', test_dataset.imgs.shape)

  return train_dataset, val_dataset, test_dataset

In [13]:
train_dataset, val_dataset, test_dataset = download_dataset('breastmnist')


Dataset source information : MedMNIST v3.0.2 @ https://github.com/MedMNIST/MedMNIST/

BreastMNIST
The BreastMNIST is based on a dataset of 780 breast ultrasound images. It is categorized into 3 classes: normal, benign, and malignant. As we use low-resolution images, we simplify the task into binary classification by combining normal and benign as positive and classifying them against malignant as negative. We split the source dataset with a ratio of 7:1:2 into training, validation and test set. The source images of 1×500×500 are resized into 1×28×28.
Using downloaded and verified file: /Users/I569354/.medmnist/breastmnist.npz
Using downloaded and verified file: /Users/I569354/.medmnist/breastmnist.npz
Using downloaded and verified file: /Users/I569354/.medmnist/breastmnist.npz

Training Dataset:
Dataset BreastMNIST of size 28 (breastmnist)
    Number of datapoints: 546
    Root location: /Users/I569354/.medmnist
    Split: train
    Task: binary-class
    Number of channels: 1
    Mean

In [18]:
class_labels = ['Malignant',
                'Normal/Benign']
# Check missing images in the dataset
if len(train_dataset.imgs) != 546:
  print("WARNING: Missing images in training dataset.")
  print(f"Found {len(train_dataset.imgs)}, should be 546.")
else:
  print("SUCCESS: No missing images in training dataset.")

if len(val_dataset.imgs) != 78:
  print("WARNING: Missing images in validation dataset.")
  print(f"Found {len(val_dataset.imgs)}, should be 78.")
else:
  print("SUCCESS: No missing images in validation dataset.")

if len(test_dataset.imgs) != 156:
  print("WARNING: Missing images in testing dataset.")
  print(f"Found {len(train_dataset.imgs)}, should be 156.")
else:
  print("SUCCESS: No missing images in test dataset.")

SUCCESS: No missing images in training dataset.
SUCCESS: No missing images in validation dataset.
SUCCESS: No missing images in test dataset.


In [19]:
# Normalising images in the dataset
# Pixel values lie between 0-255, 
# division by 255 will scale the values between 0 and 1
train_dataset.imgs = train_dataset.imgs/255.0 
val_dataset.imgs = val_dataset.imgs/255.0
test_dataset.imgs = test_dataset.imgs/255.0

In [47]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout

model = Sequential()

model.add(Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding="same"))

model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1,1),padding="same"))

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/Users/I569354/Desktop/Praxisarbeiten/Studienarbeit_Sammlung/MedicaldeepLearning/.venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [44]:
# Class imbalance handling through calculation of class weights

from sklearn.utils import class_weight
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

breast_class_weights = class_weight.compute_class_weight('balanced',
                                                         classes = np.unique(train_dataset.labels[:,0]),
                                                         y = train_dataset.labels[:, 0])

weights = { 0 : breast_class_weights[0], 1 : breast_class_weights[1] }
print(f"Class weights for imbalance {weights}")



print("Preprocessing and augmentation of images with Standardisation, Rotation and Horizontal Flips..")
datagen = ImageDataGenerator(rotation_range=10,
                             horizontal_flip=True)
my_callbacks = [tf.keras.callbacks.EarlyStopping(patience=10)]

Class weights for imbalance {0: 1.8571428571428572, 1: 0.6842105263157895}
Preprocessing and augmentation of images with Standardisation, Rotation and Horizontal Flips..


In [45]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [46]:
history = model.fit(datagen.flow(train_dataset.imgs, train_dataset.labels),
                    epochs=100,
                    callbacks=my_callbacks,
                    validation_data=(val_dataset.imgs, val_dataset.labels),
                    shuffle=True,
                    class_weight=weights)

Epoch 1/100


/Users/I569354/Desktop/Praxisarbeiten/Studienarbeit_Sammlung/MedicaldeepLearning/.venv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-10-24 12:02:12.502906: W tensorflow/core/framework/op_kernel.cc:1828] INVALID_ARGUMENT: TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was (tf.float32, tf.uint8, tf.float32), but the yielded element was (array([[[[0.6187597 ],
         [0.6087174 ],
         [0.6042217 ],
         ...,
         [0.68128   ],
         [0.6775488 ],
         [0.6745463 ]],

        [[0.6067892 ],
         [0.6084383 ],
         [0.59418255],
         ...,
         [0.5349604 ],
         [0.5324024 ],


InvalidArgumentError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was (tf.float32, tf.uint8, tf.float32), but the yielded element was (array([[[[0.6187597 ],
         [0.6087174 ],
         [0.6042217 ],
         ...,
         [0.68128   ],
         [0.6775488 ],
         [0.6745463 ]],

        [[0.6067892 ],
         [0.6084383 ],
         [0.59418255],
         ...,
         [0.5349604 ],
         [0.5324024 ],
         [0.5253816 ]],

        [[0.5271776 ],
         [0.59249705],
         [0.5684988 ],
         ...,
         [0.4987047 ],
         [0.546794  ],
         [0.56442624]],

        ...,

        [[0.08235294],
         [0.08132268],
         [0.0766023 ],
         ...,
         [0.08235294],
         [0.08169755],
         [0.07983821]],

        [[0.08206154],
         [0.08103128],
         [0.07

2024-10-24 12:02:12.539993: W tensorflow/core/framework/op_kernel.cc:1828] INVALID_ARGUMENT: TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was (tf.float32, tf.uint8, tf.float32), but the yielded element was (array([[[[0.38772213],
         [0.39752835],
         [0.411318  ],
         ...,
         [0.50597423],
         [0.5058824 ],
         [0.5133844 ]],

        [[0.29846188],
         [0.30119514],
         [0.3152473 ],
         ...,
         [0.5058824 ],
         [0.5067285 ],
         [0.51295817]],

        [[0.53477526],
         [0.50812346],
         [0.45982304],
         ...,
         [0.43455455],
         [0.44185168],
         [0.46788388]],

        ...,

        [[0.09710129],
         [0.09523139],
         [0.10615807],
         ...,
         [0.25623882],
         [0.26159796],
         [0.26139504]],

        [[0.09765554],
         [0.09104216],
         [0.10403273],
         ...,
         [0.26119